In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

## @params: [JOB_NAME]
args = getResolvedOptions(sys.argv, ['ETL-Green-Car-Data-2019'])

sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['ETL-Green-Car-Data-2019'], args)

## setup a data source and it's context name

merged_raw_datasource = glueContext.create_dynamic_frame.from_catalog(database = "db_tlc_etl_green_car", table_name = "green", transformation_ctx = "merged_raw_datasource")

## set fields mapping schema for the merged original data & drop some fields & change some data type
## lpep_pickup_datetime	pickup_date	timestamp
## lpep_dropoff_datetime	dropoff_date	timestamp
datasource_with_mappingshema = ApplyMapping.apply(frame = merged_raw_datasource, mappings = [ \
    ("vendorid", "long", "vendorid", "long"), 
    ("lpep_pickup_datetime", "string", "lpep_pickup_datetime", "timestamp"), ("lpep_dropoff_datetime", "string", "lpep_dropoff_datetime", "timestamp"), 
    ("ratecodeid", "long", "ratecodeid", "long"), 
    ("pulocationid", "long", "pulocationid", "long"), ("dolocationid", "long", "dolocationid", "long"), 
    ("passenger_count", "long", "passenger_count", "long"), ("trip_distance", "double", "trip_distance", "double"), 
    ("fare_amount", "double", "fare_amount", "double"), 
    ("extra", "double", "extra", "double"), ("mta_tax", "double", "mta_tax", "double"), 
    ("tip_amount", "double", "tip_amount", "double"), ("tolls_amount", "double", "tolls_amount", "double"), 
    ("improvement_surcharge", "double", "improvement_surcharge", "double"), 
    ("total_amount", "double", "total_amount", "double"), ("payment_type", "long", "payment_type", "long"), 
    ("trip_type", "long", "trip_type", "long")], transformation_ctx = "datasource_with_mappingshema")

## do some polishing with the mappinged source
## step 1 apply options for resolving processing to return a new table
## choice <make_struct> :we create new data storage in s3
dataframe_with_resolvechoice_appliyed = ResolveChoice.apply(frame = datasource_with_mappingshema, choice = "make_struct", transformation_ctx = "dataframe_with_resolvechoice_appliyed")

## step 2 Drop Null and NaN Fields
## @inputs: [frame = dataframe_with_resolvechoice_appliyed]
dataframe_without_nullfields = DropNullFields.apply(frame = dataframe_with_resolvechoice_appliyed, transformation_ctx = "dataframe_without_nullfields")

'''
## step 3
## Drop Some Other Fields && add new collum

custom_sparkDF = dataframe_without_nullfields.toDF()
 
#add a new column for "time_dration"
custom_sparkDF = custom_sparkDF.withColumn('time_dration',unix_timestamp(f.col('lpep_dropoff_datetime'))- unix_timestamp(f.col('lpep_pickup_datetime')))

# Convert back to a Glue DynamicFrame for further processing.
dataframe_without_nullfields = DynamicFrame.fromDF(customDF, glueContext, "custom_sparkDF")

## drop three more fields
## dataframe_without_nullfields = dataframe_without_nullfields.drop_fields(['improvement_surcharge','tolls_amount','mta_tax']) \

'''

## write target data to s3 or other data storage
# set format :parquet
datasink = glueContext.write_dynamic_frame.from_options(frame = dataframe_without_nullfields, 
                                                         connection_type = "s3", 
                                                         connection_options = {"path": "s3://data-etl-o-target-0/result"}, 
                                                         format = "parquet", 
                                                         transformation_ctx = "datasink")
## commit job and carry it out
job.commit()

In [ ]:
rulesult:
waitig to finish ,which will result target files
s3://data-etl-o-target-0/result/*.parquet

In [ ]:
'''
## step 3
## Drop Some Other Fields && add new collum

custom_sparkDF = dataframe_without_nullfields.toDF()
 
#add a new column for "time_dration"
custom_sparkDF = custom_sparkDF.withColumn('time_dration',unix_timestamp(f.col('lpep_dropoff_datetime'))- unix_timestamp(f.col('lpep_pickup_datetime')))

# Convert back to a Glue DynamicFrame for further processing.
dataframe_without_nullfields = DynamicFrame.fromDF(customDF, glueContext, "custom_sparkDF")

## drop three more fields
## dataframe_without_nullfields = dataframe_without_nullfields.drop_fields(['improvement_surcharge','tolls_amount','mta_tax']) \

'''